In [ ]:
%matplotlib inline

# Load the machine
from os.path import join
from pyleecan.Functions.load import load
from pyleecan.definitions import DATA_DIR
#from util.simulation import *
import itertools
import matplotlib.pyplot as plt
from numpy.fft import fft, fftfreq
import numpy as np
from copy import deepcopy
machine = load(join(DATA_DIR, "Machine", "Toyota_Prius.json"))

# In Jupyter notebook, we set is_show_fig=False to skip call to fig.show() to avoid a warning message
# All plot methods return the corresponding matplotlib figure and axis to further edit the resulting plot
fig, ax = machine.plot(is_show_fig=False)
plt.show()

In [ ]:
# Liste des attributs de l'aimant dans le premier trou
attrs = dir(machine.rotor.hole[0].magnet_0)
print(attrs)

In [ ]:
# Récupération de la hauteur d'aimant initiale
Lmag_ref = machine.rotor.hole[0].magnet_0.Lmag
print("Hauteur aimant originale :", Lmag_ref)

In [ ]:
import os
# Dossier de sauvegarde
save_dir = "./machines_magnet_height/"
os.makedirs(save_dir, exist_ok=True)

variations de hauteur d'aimant (Lmag) de 50% à 150% pour la creation de 10 ratios

In [ ]:
ratios = np.linspace(0.5, 1.5, 10)

In [ ]:
for i, ratio in enumerate(ratios):
    machine_mod = deepcopy(machine)
    new_Lmag = Lmag_ref * ratio
    machine_mod.rotor.hole[0].magnet_0.Lmag = new_Lmag
    machine_mod.name = f"Toyota_Prius_Lmag_{int(ratio*100)}p"

    # Sauvegarde
    file_path = os.path.join(save_dir, machine_mod.name + ".json")
    machine_mod.save(file_path)
    
    print(f"Machine {i+1}: Lmag = {new_Lmag:.5f} m → {machine_mod.name}.json")

In [ ]:
----------------------------------------------------------------------------------------------------------------------------

---------------------------Simulation-----------------------------------------

In [ ]:
%matplotlib inline

# Load the machine
from os.path import join
from pyleecan.Functions.load import load
from pyleecan.definitions import DATA_DIR

IPMSM_A = load("machines_magnet_height/Toyota_Prius_Lmag_150p.json")
# In Jupyter notebook, we set is_show_fig=False to skip call to fig.show() to avoid a warning message
# All plot methods return the corresponding matplotlib figure and axis to further edit the resulting plot
fig, ax = IPMSM_A.plot(is_show_fig=False)
plt.show()

In [ ]:
print(IPMSM_A.stator.winding.wind_mat)

In [ ]:
from os.path import join

from numpy import ones, pi, array, linspace, cos, sqrt

from pyleecan.Classes.Simu1 import Simu1
from pyleecan.Classes.InputCurrent import InputCurrent
from pyleecan.Classes.OPdq import OPdq
from pyleecan.Classes.MagFEMM import MagFEMM

# Create the Simulation
simu_femm = Simu1(name="FEMM_simulation", machine=IPMSM_A)   
# simu_femm.path_result = "path/to/folder" Path to the Result folder to use (will contain FEMM files)
p = simu_femm.machine.stator.winding.p
qs = simu_femm.machine.stator.winding.qs

# Defining Simulation Input
simu_femm.input = InputCurrent()

# Rotor speed [rpm]
N0 = 3000 
simu_femm.input.OP = OPdq(N0=N0)

# time discretization [s]
time = linspace(start=0, stop=60/N0, num=32*p, endpoint=False) # 32*p timesteps
simu_femm.input.time = time 

# Angular discretization along the airgap circonference for flux density calculation
simu_femm.input.angle = linspace(start = 0, stop = 2*pi, num=2048, endpoint=False) # 2048 steps 

# Stator currents as a function of time, each column correspond to one phase [A]
I0_rms = 250/sqrt(2) 
felec = p * N0 /60 # [Hz]
rot_dir = simu_femm.machine.stator.comp_mmf_dir()
Phi0 = 140*pi/180  # Maximum Torque Per Amp

Ia = (
    I0_rms
    * sqrt(2)
    * cos(2 * pi * felec * time + 0 * rot_dir * 2 * pi / qs + Phi0)
)
Ib = (
    I0_rms
    * sqrt(2)
    * cos(2 * pi * felec * time + 1 * rot_dir * 2 * pi / qs + Phi0)
)
Ic = (
    I0_rms
    * sqrt(2)
    * cos(2 * pi * felec * time + 2 * rot_dir * 2 * pi / qs + Phi0)
)
simu_femm.input.Is = array([Ia, Ib, Ic]).transpose()

In [ ]:
from pyleecan.Classes.MagFEMM import MagFEMM

simu_femm.mag = MagFEMM(
    type_BH_stator=0, # 0 to use the material B(H) curve, 
                      # 1 to use linear B(H) curve according to mur_lin,
                      # 2 to enforce infinite permeability (mur_lin =100000)
    type_BH_rotor=0,  # 0 to use the material B(H) curve, 
                      # 1 to use linear B(H) curve according to mur_lin,
                      # 2 to enforce infinite permeability (mur_lin =100000)
    file_name = "", # Name of the file to save the FEMM model
    is_fast_draw=True,  # Speed-up drawing of the machine by using lamination periodicity
    is_sliding_band=True,  # True to use the symetry of the lamination to draw the machine faster
    is_calc_torque_energy=True, # True to calculate torque from integration of energy derivate over rotor elements
    T_mag=60,  # Permanent magnet temperature to adapt magnet remanent flux density [°C]
    is_remove_ventS=False,  # True to remove stator ventilation duct
    is_remove_ventR=False,  # True to remove rotor ventilation duct
)

# Only the magnetic module is defined
simu_femm.elec = None
simu_femm.force = None
simu_femm.struct = None 

In [ ]:
simu_femm.mag.is_periodicity_a=True

In [ ]:
simu_femm.mag.is_periodicity_t=True

In [ ]:
simu_femm.mag.nb_worker = 4  # Number of FEMM instances to run at the same time (1 by default)

In [ ]:
simu_femm.mag.is_get_meshsolution = True # To get FEA mesh for latter post-procesing
simu_femm.mag.is_save_meshsolution_as_file = False # To save FEA results in a dat file

In [ ]:
out_femm = simu_femm.run()

In [ ]:
# Radial magnetic flux 
out_femm.mag.B.plot_2D_Data("angle","time[1]",component_list=["radial"], is_show_fig=False)
out_femm.mag.B.plot_2D_Data("wavenumber=[0,76]","time[1]",component_list=["radial"], is_show_fig=False)

In [ ]:
# Tangential magnetic flux 
out_femm.mag.B.plot_2D_Data("angle","time[1]",component_list=["tangential"], is_show_fig=False)
out_femm.mag.B.plot_2D_Data("wavenumber=[0,76]","time[1]",component_list=["tangential"], is_show_fig=False)

In [ ]:
out_femm.mag.Tem.plot_2D_Data("time", is_show_fig=False)

In [ ]:
print(out_femm.mag.Tem.values.shape)
print(simu_femm.input.Nt_tot)

In [ ]:
out_femm.mag.meshsolution.plot_contour(label="B", group_names="stator core", clim=[0,3])



In [ ]:
#%run -m pip install plotly # Uncomment this line to install plotly
import plotly.graph_objects as go
from plotly.offline import init_notebook_mode 
init_notebook_mode() 

result = out_femm.mag.B.components["radial"].get_along("angle{°}", "time")
x = result["angle"]
y = result["time"]
z = result["B_{rad}"]
fig = go.Figure(data=[go.Surface(z=z, x=x, y=y)])
fig.update_layout( )
fig.update_layout(title='Radial flux density in the airgap over time and angle',
                  autosize=True,
                  scene = dict(
                      xaxis_title='Angle [°]',
                      yaxis_title='Time [s]',
                      zaxis_title='Flux [T]'
                  ),
                  width=700,
                  margin=dict(r=20, b=100, l=10, t=100),
                 )

fig.show(config = {"displaylogo":False})

--------------------------------------------------------------------------------------------------------------------------------------------------------------